In [ ]:
!sudo apt-get update
!sudo apt-get install -y software-properties-common
!sudo apt-add-repository --yes --update ppa:ansible/ansible
!sudo apt-get install -y ansible
!curl -fsSL https://apt.releases.hashicorp.com/gpg | sudo apt-key add -
!sudo apt-add-repository "deb [arch=amd64] https://apt.releases.hashicorp.com $(lsb_release -cs) main"
!sudo apt-get update && sudo apt-get install terraform

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,372 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,745 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=qWGIGCWv8onkRYEE187ymNCUIN4H55&prompt=consent&token_

# Atividade 1 - Provisionamento de Infraestrutura Automatizada

In [ ]:
# Criar um bucket no GCP

%%writefile main.tf
provider "google" {
  project = "absolute-vertex-451423-n7"
  region  = "us-central1"
}

resource "google_storage_bucket" "petadota_imagens" {
  name     = "petadota-imagens-bucket"
  location = "US"
}

Overwriting main.tf


In [ ]:
%%writefile main.tf

provider "google" {
  project = "absolute-vertex-451423-n7"
  region  = "us-central1"
}

resource "google_storage_bucket" "petadota_imagens" {
  name          = "petadota-imagens-bucket"
  location      = "US"
  storage_class = "STANDARD"  # Alternativas: MULTI_REGIONAL, NEARLINE, COLDLINE, ARCHIVE

  # Habilita controle de versões para recuperação de arquivos excluídos
  versioning {
    enabled = true
  }

  # Define regras de retenção (Exemplo: 30 dias)
  lifecycle_rule {
    action {
      type = "Delete"
    }
    condition {
      age = 30
    }
  }

  # Define acesso público (cuidado com dados sensíveis)
  uniform_bucket_level_access = true

  labels = {
    environment = "development"
    owner       = "rodrigo"
  }
}


Overwriting main.tf


In [ ]:
!ssh-keygen -t rsa -b 4096 -C "ratolasombra@gmail.com"

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:pRQoO6RoKpEEHv6toikHpELwoOM8XGukoYrDyX/3m8w ratolasombra@gmail.com
The key's randomart image is:
+---[RSA 4096]----+
|o.     ..        |
|=.. o .  .       |
|+B o o  . .      |
|*=+o+  . o       |
|@o=..o  S        |
|B* o.            |
|Bo+.             |
|*=o  . .o .      |
|+o... . .E.      |
+----[SHA256]-----+


In [ ]:
cat ~/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDhkJhIQX6dHf7zEKfnB5iCNIBZHe3tde7qUAYZHQntL//EvfgaL0oO2nvifVvnVMFyji8JA7x8+zNCoq1mehsrtPPMS97/XuF+DRNLG4VobRvPVzs3x2R/6YmR8whkfvUfdEzOp5V5Ip9NWf+pFpMRbSJrK2JGebBTP+iuf+iLFXIMiHkNz6HDS9VM51M4b7zRMJWFquZA9clN4ZjVNxdcceBJA1PlxsMo+YYvYWt6WNTqa4MvEkYGmGyqrt1I6mOcFCFW5M2BmwTIJfPQObKfXXXTtPhaUsL6IYaizFm8rNZ74hqsS/P4SKEqBuJoaiBl2QZmboULpRjl6R6azWSWr8VwY7oogEG6vjk8PV+Kxg91kS2M67wfChBcuMFYYHdWMaTRv9pKfq6PklJqHRYhrej3TtXdwV3lZP3z0JtzwlWERriZI5RVggp0Zh4J1+v7My4r1YJoOXqE1OLDLUPDpvTzeq1VqegPwvJldyWR+O3e7babUmwGH3ovZoBA7ZV8Tb78Wb8RmYoQgUf+EizbzGXUxT5sX7IRV+ltupFUpg3sLGMz8Nl1ZqTJFXy32M+WfmCHYw7ejxaVJr57RLookFqWaD8y69kLM0827T3fxiqB8guYcpAGSPUWhG+2CaM2gGzDRJt6638+vVMkaGt2bdQlGjHdBEWbaZBw1xPGXQ== ratolasombra@gmail.com


In [ ]:
# Provisionar uma máquina virtual Linux

%%writefile -a main.tf
resource "google_compute_instance" "db_vm" {
  name         = "petadota-db-vm"
  machine_type = "e2-medium"
  zone         = "us-central1-a"

  boot_disk {
    initialize_params {
      image = "debian-cloud/debian-11"
    }
  }

  network_interface {
    network = "default"
    access_config {
    }
  }

  metadata = {
    ssh-keys = "ubuntu:ssh-rsa ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDhkJhIQX6dHf7zEKfnB5iCNIBZHe3tde7qUAYZHQntL//EvfgaL0oO2nvifVvnVMFyji8JA7x8+zNCoq1mehsrtPPMS97/XuF+DRNLG4VobRvPVzs3x2R/6YmR8whkfvUfdEzOp5V5Ip9NWf+pFpMRbSJrK2JGebBTP+iuf+iLFXIMiHkNz6HDS9VM51M4b7zRMJWFquZA9clN4ZjVNxdcceBJA1PlxsMo+YYvYWt6WNTqa4MvEkYGmGyqrt1I6mOcFCFW5M2BmwTIJfPQObKfXXXTtPhaUsL6IYaizFm8rNZ74hqsS/P4SKEqBuJoaiBl2QZmboULpRjl6R6azWSWr8VwY7oogEG6vjk8PV+Kxg91kS2M67wfChBcuMFYYHdWMaTRv9pKfq6PklJqHRYhrej3TtXdwV3lZP3z0JtzwlWERriZI5RVggp0Zh4J1+v7My4r1YJoOXqE1OLDLUPDpvTzeq1VqegPwvJldyWR+O3e7babUmwGH3ovZoBA7ZV8Tb78Wb8RmYoQgUf+EizbzGXUxT5sX7IRV+ltupFUpg3sLGMz8Nl1ZqTJFXy32M+WfmCHYw7ejxaVJr57RLookFqWaD8y69kLM0827T3fxiqB8guYcpAGSPUWhG+2CaM2gGzDRJt6638+vVMkaGt2bdQlGjHdBEWbaZBw1xPGXQ== ratolasombra@gmail.com"
  }
}

output "vm_ip" {
  value = google_compute_instance.db_vm.network_interface[0].access_config[0].nat_ip
}

Appending to main.tf


In [ ]:
!terraform init
!terraform apply -auto-approve

Initializing the backend...
Initializing provider plugins...
- Finding latest version of hashicorp/google...
- Installing hashicorp/google v6.25.0...
- Installed hashicorp/google v6.25.0 (signed by HashiCorp)
Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.

Terraform used the selected providers to generate the following execution plan. R

In [ ]:
%%writefile setup_mysql.yml
- hosts: all
  become: yes
  tasks:
    - name: Install MySQL
      apt:
        name: mysql-server
        state: present

    - name: Start MySQL service
      service:
        name: mysql
        state: started
        enabled: yes

    - name: Create Pet_Adota database
      mysql_db:
        name: Pet_Adota
        state: present

    - name: Create animais_disponiveis table
      mysql_query:
        query: |
          CREATE TABLE animais_disponiveis (
            id INT AUTO_INCREMENT PRIMARY KEY,
            raca VARCHAR(255) NOT NULL,
            url VARCHAR(255) NOT NULL,
            tipo ENUM('cachorro', 'gato') NOT NULL
          );

    - name: Create read-only user
      mysql_user:
        name: 'read_user'
        password: 'readonlypass'
        priv: 'Pet_Adota.*:SELECT'
        state: present

Writing setup_mysql.yml


In [ ]:
!terraform init
!terraform apply -auto-approve

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of hashicorp/google from the dependency lock file
- Using previously-installed hashicorp/google v6.25.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
google_storage_bucket.petadota_imagens: Refreshing state... [id=petadota-imagens-bucket]
google_compute_instance.db_vm: Refreshing state... [id=projects/absolute-vertex-451423-n7/zones/us-central1-a/instances/petadota-db-vm]

No changes. Your infrastructure matches the configuration.

Terraform has compared your real infrastructure against your configuration and found 

In [ ]:
!ssh-keyscan -H 34.31.170.11 >> ~/.ssh/known_hosts

# 34.31.170.11:22 SSH-2.0-OpenSSH_8.4p1 Debian-5+deb11u4
# 34.31.170.11:22 SSH-2.0-OpenSSH_8.4p1 Debian-5+deb11u4
# 34.31.170.11:22 SSH-2.0-OpenSSH_8.4p1 Debian-5+deb11u4
# 34.31.170.11:22 SSH-2.0-OpenSSH_8.4p1 Debian-5+deb11u4
# 34.31.170.11:22 SSH-2.0-OpenSSH_8.4p1 Debian-5+deb11u4


In [ ]:
!ansible-playbook -i '34.31.170.11' setup_mysql.yml -u ubuntu --private-key ~/.ssh/id_rsa

/usr/lib/python3/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
/usr/lib/python3/dist-packages/paramiko/transport.py:261: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
[WARNING]: Unable to parse /content/34.31.170.11 as an inventory source
[WARNING]: No inventory was parsed, only implicit localhost is available
[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit

#Atividade 2 - Pipeline de Imagens da API para o Banco

In [ ]:
import requests

dog_api_url = "https://dog.ceo/api/breeds/image/random"
dog_images = []

for i in range(10):
    response = requests.get(dog_api_url)
    dog_images.append(response.json()['message'])

cat_api_url = "https://api.thecatapi.com/v1/images/search"
cat_images = []

for i in range(10):
    response = requests.get(cat_api_url)
    cat_images.append(response.json()[0]['url'])

dog_images[:5], cat_images[:5]

(['https://images.dog.ceo/breeds/pointer-germanlonghair/hans4.jpg',
  'https://images.dog.ceo/breeds/terrier-welsh/lucy.jpg',
  'https://images.dog.ceo/breeds/terrier-westhighland/n02098286_4876.jpg',
  'https://images.dog.ceo/breeds/terrier-border/n02093754_5029.jpg',
  'https://images.dog.ceo/breeds/danish-swedish-farmdog/ebba_004.jpg'],
 ['https://cdn2.thecatapi.com/images/_GVSjXZ4i.jpg',
  'https://cdn2.thecatapi.com/images/16k.jpg',
  'https://cdn2.thecatapi.com/images/4s9.jpg',
  'https://cdn2.thecatapi.com/images/c2p.jpg',
  'https://cdn2.thecatapi.com/images/asg.jpg'])

In [ ]:
!gcloud auth application-default set-quota-project absolute-vertex-451423-n7


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "absolute-vertex-451423-n7" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [ ]:
import json
from google.colab import files
uploaded = files.upload()

Saving absolute-vertex-451423-n7-54a62dba8bbd.json to absolute-vertex-451423-n7-54a62dba8bbd (2).json


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/absolute-vertex-451423-n7-54a62dba8bbd.json"

In [ ]:
from google.cloud import storage

client = storage.Client(project="absolute-vertex-451423-n7")
bucket = client.get_bucket('petadota-imagens-bucket')

In [ ]:
from google.cloud import storage
import os
import requests

client = storage.Client()
bucket = client.get_bucket('petadota-imagens-bucket')

def download_and_store_image(image_url, animal_type, breed, counter):
    filename = f"{breed}_{str(counter).zfill(2)}.jpg"
    file_path = f"{animal_type}/{breed}/{filename}"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(image_url, headers=headers)
    if response.status_code == 200:
        with open(filename, 'wb') as out_file:
            out_file.write(response.content)
    else:
        raise Exception(f"Failed to download image. Status code: {response.status_code}")

    blob = bucket.blob(file_path)
    blob.upload_from_filename(filename)

    os.remove(filename)

    return file_path

dog_image_paths = []
for idx, dog_url in enumerate(dog_images):
    breed = dog_url.split('/')[4]
    path = download_and_store_image(dog_url, 'cachorro', breed, idx+1)
    dog_image_paths.append(path)

cat_image_paths = []
for idx, cat_url in enumerate(cat_images):
    breed = cat_url.split('/')[4]
    path = download_and_store_image(cat_url, 'gato', breed, idx+1)
    cat_image_paths.append(path)

print(dog_image_paths, cat_image_paths)

['cachorro/pointer-germanlonghair/pointer-germanlonghair_01.jpg', 'cachorro/terrier-welsh/terrier-welsh_02.jpg', 'cachorro/terrier-westhighland/terrier-westhighland_03.jpg', 'cachorro/terrier-border/terrier-border_04.jpg', 'cachorro/danish-swedish-farmdog/danish-swedish-farmdog_05.jpg', 'cachorro/spaniel-sussex/spaniel-sussex_06.jpg', 'cachorro/sheepdog-english/sheepdog-english_07.jpg', 'cachorro/pitbull/pitbull_08.jpg', 'cachorro/spaniel-sussex/spaniel-sussex_09.jpg', 'cachorro/bluetick/bluetick_10.jpg'] ['gato/_GVSjXZ4i.jpg/_GVSjXZ4i.jpg_01.jpg', 'gato/16k.jpg/16k.jpg_02.jpg', 'gato/4s9.jpg/4s9.jpg_03.jpg', 'gato/c2p.jpg/c2p.jpg_04.jpg', 'gato/asg.jpg/asg.jpg_05.jpg', 'gato/me56sI74P.jpg/me56sI74P.jpg_06.jpg', 'gato/e0b.jpg/e0b.jpg_07.jpg', 'gato/MTY0NjU4Mg.jpg/MTY0NjU4Mg.jpg_08.jpg', 'gato/a4g.jpg/a4g.jpg_09.jpg', 'gato/9tj.jpg/9tj.jpg_10.jpg']


In [ ]:
from google.cloud import storage

def upload_image_to_gcp(image_path, breed, image_name):
    storage_client = storage.Client()

    bucket_name = "petadota-imagens-bucket"
    bucket = storage_client.get_bucket(bucket_name)

    destination_blob_name = f"pet-adota-imagens/{'cachorro' if breed != 'gato' else 'gato'}/{breed}/{image_name}"

    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(image_path)
    print(f"Image {image_name} uploaded to GCP at {destination_blob_name}")


In [ ]:
!pip install pymysql

def register_image_in_db(breed, image_name, image_url, tipo):
    connection = pymysql.connect(
        host='34.31.170.11',
        user='root',
        password='',
        database='Pet_Adota'
    )

    cursor = connection.cursor()
    query = "INSERT INTO animais_disponiveis (raca, url, tipo) VALUES (%s, %s, %s)"
    cursor.execute(query, (breed, image_url, tipo))
    connection.commit()
    cursor.close()
    connection.close()
    print(f"Metadata for {image_name} has been registered in the database.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install apache-airflow

In [ ]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta

default_args = {
    'owner': 'airflow',
    'retries': 1,
    'retry_delay': timedelta(seconds=5),
}

dag = DAG(
    'pipeline_imagens_pet_adota',
    default_args=default_args,
    description='Pipeline para baixar imagens de cães e gatos, salvar no GCP e registrar no MySQL',
    schedule_interval=timedelta(seconds=30),
    start_date=datetime(2025, 2, 28),
    catchup=False,
)

def pipeline():
    download_dog_images()
    download_cat_images()

pipeline_task = PythonOperator(
    task_id='run_pipeline',
    python_callable=pipeline,
    dag=dag,
)

pipeline_task

In [ ]:
!airflow users create \
    --username admin \
    --firstname Rodrigo \
    --lastname Charles \
    --role Admin \
    --email ratolasombra@gmail.com


Please confirm database initialize (or wait 4 seconds to skip it). Are you sure? [y/N]
[2025-03-11T23:47:57.248+0000] {db.py:965} INFO - Log template table does not exist (added in 2.3.0); skipping log template sync.
/usr/local/lib/python3.11/dist-packages/flask_limiter/extension.py:330 UserWarning: Using the in-memory storage for tracking rate limits as no storage was explicitly specified. This is not recommended for production use. See: https://flask-limiter.readthedocs.io#configuring-a-storage-backend for documentation about configuring the storage backend.
[2025-03-11T23:47:58.141+0000] {override.py:969} INFO - Security DB not found Creating all Models from Base
[2025-03-11T23:47:59.278+0000] {override.py:971} INFO - Security DB Created
[2025-03-11T23:47:59.308+0000] {override.py:1526} INFO - Inserted Role: Admin
[2025-03-11T23:47:59.314+0000] {override.py:983} WARNING - No user yet created, use flask fab command to do it.
[2025-03-11T23:47:59.462+0000] {override.py:1930} INFO - C

In [ ]:
!airflow db init
!airflow webserver -p 8081
!airflow scheduler

DB: sqlite:////root/airflow/airflow.db
[2025-03-11T23:48:45.016+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-03-11T23:48:45.017+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running stamp_revision  -> 5f2621c13b39
WARNI [airflow.models.crypto] empty cryptography key - values will not be stored encrypted.
Initialization done
[2025-03-11T23:48:48.678+0000] {configuration.py:2112} INFO - Creating new FAB webserver config file in: /root/airflow/webserver_config.py
  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8081
Timeout: 120
Logfiles: - -
Ac

#Atividade 3 - Processamento de Interações de Usuários em Tempo Real

In [1]:
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz
!tar -xzf kafka_2.13-3.6.1.tgz
!mv kafka_2.13-3.6.1 kafka

--2025-03-19 13:40:05--  https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113466638 (108M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.6.1.tgz’

kafka_2.13-3.6.1.tg 100%[===================>] 108.21M  2.63MB/s    in 57s     

2025-03-19 13:41:02 (1.91 MB/s) - ‘kafka_2.13-3.6.1.tgz’ saved [113466638/113466638]



In [2]:
!kafka/bin/zookeeper-server-start.sh -daemon kafka/config/zookeeper.properties
!kafka/bin/kafka-server-start.sh -daemon kafka/config/server.properties

In [3]:
!kafka/bin/kafka-topics.sh --create --topic interacoes_animais --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1

[2025-03-19 13:41:12,480] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available. (org.apache.kafka.clients.NetworkClient)
[2025-03-19 13:41:12,555] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available. (org.apache.kafka.clients.NetworkClient)
[2025-03-19 13:41:12,657] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available. (org.apache.kafka.clients.NetworkClient)
[2025-03-19 13:41:12,859] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available. (org.apache.kafka.clients.NetworkClient)
[2025-03-19 13:41:13,265] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broke

In [13]:
!pip install kafka-python

from kafka import KafkaProducer
import json
import time
import random

producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

racas_cachorro = ["labrador", "SRD", "poodle", "beagle", "husky"]
racas_gato = ["siames", "persa", "sphynx", "ragdoll", "bengal"]

def gerar_evento():
    usuario_id = random.randint(1, 1000)
    tipo = random.choice(["cachorro", "gato"])
    raca = random.choice(racas_cachorro if tipo == "cachorro" else racas_gato)
    timestamp = int(time.time())

    evento = {
        "usuario_id": usuario_id,
        "raca": raca,
        "tipo": tipo,
        "timestamp": timestamp
    }

    return evento

for _ in range(100):
    evento = gerar_evento()
    producer.send('interacoes_animais', evento)
    print(f"Evento publicado: {evento}")
    time.sleep(1)

Evento publicado: {'usuario_id': 999, 'raca': 'ragdoll', 'tipo': 'gato', 'timestamp': 1742392329}
Evento publicado: {'usuario_id': 927, 'raca': 'bengal', 'tipo': 'gato', 'timestamp': 1742392330}
Evento publicado: {'usuario_id': 761, 'raca': 'persa', 'tipo': 'gato', 'timestamp': 1742392331}
Evento publicado: {'usuario_id': 822, 'raca': 'husky', 'tipo': 'cachorro', 'timestamp': 1742392332}
Evento publicado: {'usuario_id': 427, 'raca': 'poodle', 'tipo': 'cachorro', 'timestamp': 1742392333}
Evento publicado: {'usuario_id': 607, 'raca': 'persa', 'tipo': 'gato', 'timestamp': 1742392334}
Evento publicado: {'usuario_id': 5, 'raca': 'labrador', 'tipo': 'cachorro', 'timestamp': 1742392335}
Evento publicado: {'usuario_id': 18, 'raca': 'husky', 'tipo': 'cachorro', 'timestamp': 1742392336}
Evento publicado: {'usuario_id': 210, 'raca': 'sphynx', 'tipo': 'gato', 'timestamp': 1742392337}
Evento publicado: {'usuario_id': 295, 'raca': 'husky', 'tipo': 'cachorro', 'timestamp': 1742392338}
Evento publicad

In [14]:
!.option("kafka.bootstrap.servers", "VM_EXTERNAL_IP:9092")

/bin/bash: -c: line 1: syntax error near unexpected token `"kafka.bootstrap.servers",'
/bin/bash: -c: line 1: `.option("kafka.bootstrap.servers", "VM_EXTERNAL_IP:9092")'


In [20]:
!pip install pyspark
!pip install kafka-python

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

# Criando a sessão do Spark com suporte ao Kafka
spark = SparkSession.builder \
    .appName("MonitoramentoAdocoes") \
    .getOrCreate()

# Definir o schema que esperamos dos eventos Kafka
schema = StructType([
    StructField("usuario_id", IntegerType(), True),
    StructField("raca", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("timestamp", LongType(), True)
])

# Consumir dados do Kafka
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "interacoes_animais") \
    .load()

# Converter o valor do Kafka de bytes para String
df_json = df_kafka.selectExpr("CAST(value AS STRING)")

# Parse do JSON para o formato estruturado
df_parsed = df_json.selectExpr("json_tuple(value, 'usuario_id', 'raca', 'tipo', 'timestamp') as (usuario_id, raca, tipo, timestamp)")

# Alterar os tipos de dados conforme o esperado
df_final = df_parsed.select(
    col("usuario_id").cast(IntegerType()),
    col("raca"),
    col("tipo"),
    col("timestamp").cast(LongType())
)

# Agora você pode realizar operações como groupBy
tendencias = df_final.groupBy("raca", "tipo").count()

# Exibir o resultado para cada micro-batch
query = tendencias.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

# Espera até que a consulta termine (se necessário)
query.awaitTermination()


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

# Criando a sessão do Spark
spark = SparkSession.builder.appName("MonitoramentoAdocoes").getOrCreate()

# Dados de entrada
data = [
    (1, "Labrador", "cachorro", 1700000000),
    (2, "Persa", "gato", 1700000020),
    (3, "Labrador", "cachorro", 1700000050),
]

# Definindo o schema, alterando o tipo de timestamp para LongType
schema = StructType([
    StructField("usuario_id", IntegerType()),
    StructField("raca", StringType()),
    StructField("tipo", StringType()),
    StructField("timestamp", LongType())  # Alterando para LongType
])

# Criando o DataFrame
df_json = spark.createDataFrame(data, schema)

# Agrupando por raça e tipo e contando as visualizações
tendencias = df_json.groupBy("raca", "tipo").agg(count("*").alias("visualizacoes"))

# Exibindo o resultado
tendencias.show()


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [23]:
from pyspark.sql.functions import col, expr
df_filtrado = df_json.filter(col("timestamp") >= expr("current_timestamp() - INTERVAL 1 MINUTE"))

df_filtrado.show()

NameError: name 'df_json' is not defined

In [ ]:
spark = SparkSession.builder \
    .appName("SalvarEstatisticasMySQL") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33") \
    .getOrCreate()

In [ ]:
!telnet 34.31.170.11 3306

/bin/bash: line 1: telnet: command not found


In [ ]:
!curl ifconfig.me

34.31.160.175

In [ ]:
!sudo ufw allow 3306/tcp
!sudo ufw reload

sudo: ufw: command not found
sudo: ufw: command not found


In [ ]:
!pip install pymysql sqlalchemy
!pip install cloud-sql-python-connector

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy

instance_connection_name = "projeto-mari-452814:us-central1:colab"
db_user = "root"
db_pass = ""

connector = Connector()

def getconn():
    conn = connector.connect(
        instance_connection_name,
        "pymysql",
        user=db_user,
        password=db_pass
    )
    return conn

pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

with pool.connect() as db_conn:
    db_conn.execute("CREATE DATABASE IF NOT EXISTS Pet_Adota")
    print("Banco de dados 'Pet_Adota' criado com sucesso!")

<ipython-input-95-ce8c90bfff9e>:30 RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)

Banco de dados 'Pet_Adota' criado com sucesso!


In [ ]:
import pandas as pd

data = [
    ("labrador", "cachorro", 100),
    ("siames", "gato", 50),
    ("poodle", "cachorro", 75),
]

df_estatisticas = pd.DataFrame(data, columns=["raca", "tipo", "visualizacoes"])
print(df_estatisticas)

       raca      tipo  visualizacoes
0  labrador  cachorro            100
1    siames      gato             50
2    poodle  cachorro             75


In [ ]:
from google.cloud.sql.connector import Connector
import pymysql
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import base

instance_connection_name = "projeto-mari-452814:us-central1:colab"
db_user = "root"
db_pass = ""
db_name = "Pet_Adota"

connector = Connector()

def getconn():
    conn = connector.connect(
        instance_connection_name,
        "pymysql",
        user=db_user,
        password=db_pass,
        db=db_name
    )
    return conn

class MySQLConnection(base.Connection):
    def __init__(self, conn):
        self.conn = conn
        self.cursor = conn.cursor()

    def execute(self, sql, params=None):
        return self.cursor.execute(sql, params)

    def close(self):
        self.cursor.close()
        self.conn.close()


def get_sqlalchemy_engine():
    return create_engine(
        "mysql+pymysql://",
        creator=lambda: MySQLConnection(getconn())
    )

pool = get_sqlalchemy_engine()

data = [
    ("labrador", "cachorro", 100),
    ("siames", "gato", 50),
    ("poodle", "cachorro", 75),
]

df_estatisticas = pd.DataFrame(data, columns=["raca", "tipo", "visualizacoes"])

df_estatisticas.to_sql(
    name="estatisticas_animais",
    con=pool,
    if_exists="append",
    index=False
)

print("Estatísticas salvas no MySQL com sucesso!")


<ipython-input-104-d9f89e9e9358>:63 UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

AttributeError: 'Engine' object has no attribute 'cursor'

In [ ]:
import pymysql
import pandas as pd

host = "35.188.171.159"
user = "root"
password = ""
database = "Pet_Adota"

data = [
    ("labrador", "cachorro", 100),
    ("siames", "gato", 50),
    ("poodle", "cachorro", 75),
]

df_estatisticas = pd.DataFrame(data, columns=["raca", "tipo", "visualizacoes"])

connection = None

try:
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    print("Conexão bem-sucedida!")

    cursor = connection.cursor()

    create_table_query = """
    CREATE TABLE IF NOT EXISTS estatisticas_animais (
        id INT AUTO_INCREMENT PRIMARY KEY,
        raca VARCHAR(255) NOT NULL,
        tipo VARCHAR(255) NOT NULL,
        visualizacoes INT NOT NULL,
        data_registro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    cursor.execute(create_table_query)
    print("Tabela 'estatisticas_animais' criada com sucesso!")

    for index, row in df_estatisticas.iterrows():
        insert_query = """
        INSERT INTO estatisticas_animais (raca, tipo, visualizacoes)
        VALUES (%s, %s, %s)
        """
        cursor.execute(insert_query, (row["raca"], row["tipo"], row["visualizacoes"]))

    connection.commit()
    print("Estatísticas salvas no MySQL com sucesso!")

except Exception as e:
    print(f"Erro ao conectar ou salvar dados no MySQL: {e}")

finally:
    if connection:
        connection.close()
        print("Conexão fechada.")

Conexão bem-sucedida!
Tabela 'estatisticas_animais' criada com sucesso!
Estatísticas salvas no MySQL com sucesso!
Conexão fechada.


In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import pandas as pd
import pymysql

def processar_tendencias():
    host = "35.188.171.159"
    user = "root"
    password = ""
    database = "Pet_Adota"

    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        print("Conexão bem-sucedida!")

        query = """
        SELECT raca, tipo, SUM(visualizacoes) AS total_visualizacoes
        FROM estatisticas_animais
        GROUP BY raca, tipo
        ORDER BY total_visualizacoes DESC;
        """
        df_tendencias = pd.read_sql(query, connection)
        print("Tendências encontradas:")
        print(df_tendencias)

    except Exception as e:
        print(f"Erro ao processar tendências: {e}")

    finally:
        if connection:
            connection.close()
            print("Conexão fechada.")

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2023, 10, 25),
    'retries': 1,
    'retry_delay': timedelta(seconds=5),
}

dag = DAG(
    'visualizar_tendencias',
    default_args=default_args,
    description='DAG para visualizar tendências a cada 30 segundos',
    schedule=timedelta(seconds=30),  # Usando 'schedule' em vez de 'schedule_interval'
    catchup=False,
)

tarefa_processar_tendencias = PythonOperator(
    task_id='processar_tendencias',
    python_callable=processar_tendencias,
    dag=dag,
)

tarefa_processar_tendencias

<Task(PythonOperator): processar_tendencias>